In [4]:
#installing necessary package
# !pip install -q transformers einops accelerate langchain bitsandbytes
# !pip install sentencepiece
# !pip install xformers

In [1]:
#import necessary libraries
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

from transformers import LlamaTokenizer, LlamaForCausalLM
from langchain import PromptTemplate,  LLMChain

In [2]:

def text_summarization():

  model_path = "openlm-research/open_llama_7b"

  tokenizer = LlamaTokenizer.from_pretrained(model_path)
  model = LlamaForCausalLM.from_pretrained(
      model_path, torch_dtype=torch.float16, device_map='auto',
  )

  pipeline1 = pipeline(
      "text-generation", #task
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.bfloat16,
      trust_remote_code=True,
      device_map="auto",
      max_length=1000,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id
  )
  llm = HuggingFacePipeline(pipeline = pipeline1, model_kwargs = {'temperature':0})
  template = """
              Write a concise summary of the following text.
              Return your response in paragraph which covers the key points of the text.
              ```{text}```
           """

  prompt = PromptTemplate(template=template, input_variables=["text"])

  llm_chain = LLMChain(prompt=prompt, llm=llm)
  # text=input("Enter the text you want to Summarize")
  text="""
  Question answering (QA) systems are a type of natural language processing (NLP) technology that provide
precise and concise answers to questions posed in natural language. These systems have the potential to
revolutionize the way we access information and can be applied in a wide range of fields including education,
customer service, and health care.There are several approaches to building QA systems, including rule-based,
information retrieval, and machine learning-based approaches. Rule-based systems rely on predefined rules and
patterns to extract answers from a given text, while information retrieval systems use search algorithms to
retrieve relevant information from a large database. Machine learning-based systems, on the other hand, use
training data to learn to extract answers from text.One of the main challenges faced by QA systems is the need
to understand the context and intent behind a question. This requires the system to have a deep understanding of
the language and the ability to make inferences based on the given information. Another challenge is the need to
extract relevant information from a large and potentially unstructured dataset.Despite these challenges, QA
systems have a wide range of applications, including education, customer service, and health care. In education,
QA systems can be used to provide personalized learning experiences and help students learn more efficiently.
In customer service, QA systems can be used to handle a high volume of queries and provide quick and accurate
responses to customers. In health care, QA systems can be used to assist doctors and patients by providing
timely and accurate information about medical conditions and treatments.Overall, this review aims to provide a
comprehensive overview of QA systems, their approaches, challenges, and applications. By understanding the
current state of development and the potential impact of QA systems, we can better utilize these technologies to
improve various industries and enhance the way we access information.
  """
  print("\n")
  print("\n")
  print("<<<<TEXT SUMMARIZATION>>>>")
  print(llm_chain.run(text))

In [3]:
text_summarization()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]





<<<<TEXT SUMMARIZATION>>>>
 
 **Explanation**
 
 The text is about QA systems which are developed in order to answer questions in natural language. These systems can be used in various fields such as education, customer service and health care etc.
 
 The text provides an overview of the various approaches used in QA systems, namely rule based, Information Retrieval and Machine Learning based approaches.
 
 Rule-based QA systems have the ability to recognize rules in the text and extract the answer to the question based on them. Information Retrieval based QA systems utilize search algorithms to retrieve relevant information from databases and Machine Learning based QA systems use training data to learn to extract the answers from text.
 
 The challenge faced by these systems is the need to understand the context and the intent behind a question. This requires the systems to have the ability to understand language and make inferences from information presented in the question. Anot

In [11]:
generated_summary="""
Question answering (QA) systems are a type of natural language processing (NLP) technology that provide
precise and concise answers to questions posed in natural language. These systems have the potential to
revolutionize the way we access information and can be applied in a wide range of fields including education,
customer service, and health care.


One of the main challenges faced by QA systems is the need to understand the context and intent behind a question. This requires the system to have a deep understanding of the language and the ability to make inferences based on the given information. Another challenge is the need to extract relevant information from a large and potentially unstructured dataset.
Despite these challenges, QA systems have a wide range of applications, including education, customer service, and health care. In education, QA systems can be used to provide personalized learning experiences and help students
"""
len(generated_summary)

949

In [12]:

reference_summaries="""Question answering (QA) systems are a type of natural language processing (NLP) technology that provide
precise and concise answers to questions posed in natural language. rule-based,
information retrieval, and machine learning-based are different approaches in building QA systems.
One of the main challenges faced by QA systems is the need
to understand the context and intent behind a question. Another challenge is the need to
extract relevant information from a large and potentially unstructured dataset. QA
systems have a wide range of applications, including education, customer service, and health care. By understanding the
current state of development and the potential impact of QA systems, we can better utilize these technologies to
improve various industries and enhance the way we access information."""

In [13]:
len(reference_summaries)

817

In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [8]:
! pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from rouge import Rouge

def calculate_rouge_scores(generated_summary, reference_summaries):
    rouge = Rouge()
    scores = rouge.get_scores(generated_summary, reference_summaries)
    rouge_1_score = scores[0]["rouge-1"]["f"]
    rouge_2_score = scores[0]["rouge-2"]["f"]
    rouge_l_score = scores[0]["rouge-l"]["f"]
    return rouge_1_score, rouge_2_score, rouge_l_score


In [15]:

rouge_1, rouge_2, rouge_l = calculate_rouge_scores(generated_summary, reference_summaries)

print(f"ROUGE-1 Score: {rouge_1}")
print(f"ROUGE-2 Score: {rouge_2}")
print(f"ROUGE-L Score: {rouge_l}")

ROUGE-1 Score: 0.7441860415143321
ROUGE-2 Score: 0.614107878831976
ROUGE-L Score: 0.7325581345375881
